In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Test DFA-2-RegExp

In [ ]:
// 06-Test-DFA-2-RegExp.ts
import { instance } from "@viz-js/viz";
import { RecursiveSet } from "recursive-set";
import {
    State,
    Char,
    DFA,
    RegExp,
    TransRelDet,
    key,
    dfa2regexp,
} from "./05-DFA-2-RegExp";
// Hinzugefügt: Import von dfa2dot
import { dfa2dot } from "./FSM-2-Dot";

In [ ]:
// Singletons {1}, {2}, {3}
// Wir definieren explizit den Typ <State>, damit TS weiß, was drin ist.
const S0 = new RecursiveSet<State>(1);
const S1 = new RecursiveSet<State>(2);
const S2 = new RecursiveSet<State>(3);

// Q = { {1}, {2}, {3} }
// Hier definieren wir, dass das Set andere RecursiveSets enthält.
const Q = new RecursiveSet<RecursiveSet<State>>(S0, S1, S2);

// Σ = {a, b}
const Sigma = new RecursiveSet<Char>("a", "b");

// δ (Übergangsfunktion)
const delta: TransRelDet = new Map<string, RecursiveSet<State>>();
delta.set(key(S0, "a"), S1); // (1,a)->2
delta.set(key(S1, "b"), S2); // (2,b)->3
delta.set(key(S2, "a"), S1); // (3,a)->2

// Startzustand und akzeptierende Zustände
const q0 = S0;
const A = new RecursiveSet<RecursiveSet<State>>(S2);

// DFA Objekt
// Jetzt passen alle Typen zur Definition von DFA in 05-DFA-2-RegExp.ts
const dfa: DFA = { Q, Sigma, delta, q0, A };

In [ ]:
const { dot } = dfa2dot(dfa);
const viz = await instance();
display.html(viz.renderString(dot, { format: "svg" }));

In [ ]:
function prettyPrintRegExp(r: RegExp): string {
    if (typeof r === 'number' || typeof r === 'string') {
        return typeof r === 'string' ? `'${r}'` : String(r);
    }

    if (Array.isArray(r)) {
        // Typ-Casting zu any[] um TS-Warnungen bei map zu vermeiden
        const elements = (r as any[]).map((el: any) => prettyPrintRegExp(el)).join(", ");
        return `(${elements})`;
    }
    
    return String(r);
}

// Berechnung des regulären Ausdrucks aus dem DFA
const r: RegExp = dfa2regexp(dfa);
console.log("Unvereinfachter regulärer Ausdruck:");
console.log(prettyPrintRegExp(r));

## Helpfunctions:

In [ ]:
function isZero(r: RegExp): boolean {
  return r === 0;
}
function isEps(r: RegExp): boolean {
  return r === "ε" || r === "𝜀";
}
function eq(a: RegExp, b: RegExp): boolean {
  return JSON.stringify(a) === JSON.stringify(b);
}

As this regular expression is nearly unreadable,  The notebook `Rewrite.ipynb` contains the definition of the function `simplify` that can be used to simplify this expression.

In [ ]:
function simplify(r: RegExp): RegExp {
  if (typeof r === "string" || typeof r === "number") return r;
  if (!Array.isArray(r)) return r;

  // ---------- Kleene Star ----------
  if (r.length === 2 && r[1] === "*") {
    // Zugriff auf das innere Element r[0]
    // Da RegExp rekursiv definiert ist, rufen wir simplify rekursiv auf.
    // Wir müssen hier casten, da TS bei Tupeln manchmal pingelig ist.
    let inner = simplify(r[0] as RegExp);

    // 0* → ε
    if (isZero(inner)) return "ε";   

    // ε* → ε
    if (isEps(inner)) return "ε";    

    // (r*)* → r*
    if (Array.isArray(inner) && inner.length === 2 && inner[1] === "*") {
      return inner;
    }

    // (ε + r)* → r*
    if (
      Array.isArray(inner) &&
      inner.length === 3 &&
      inner[1] === "+" &&
      (isEps(inner[0] as RegExp) || isZero(inner[0] as RegExp))
    ) {
      const r2 = simplify(inner[2] as RegExp);
      return [r2, "*"];
    }

    return [inner, "*"];
  }

  // ---------- Binäre Operationen (+ oder ⋅) ----------
  // Cast auf [RegExp, BinaryOp, RegExp]
  if (r.length === 3) {
      const left = simplify(r[0] as RegExp);
      const op = r[1];
      const right = simplify(r[2] as RegExp);

      // ------------------ +
      if (op === "+") {
        if (isZero(left)) return right;
        if (isZero(right)) return left;
        if (eq(left, right)) return left;
        return [left, "+", right];
      }

      // ------------------ ⋅
      if (op === "⋅") {
        if (isZero(left) || isZero(right)) return 0;
        if (isEps(left)) return right;
        if (isEps(right)) return left;
        return [left, "⋅", right];
      }
  }

  return r;
}

In [ ]:
let s = simplify(r);
s = simplify(s); 

console.log("Vereinfachter Ausdruck (Struktur):");
console.log(prettyPrintRegExp(s));

The function `regexp_2_string` takes a regular expression that is represented as a nested tuple and transforms it into a string.

In [ ]:
function regexpToString(r: RegExp): string {
  // leere Sprache
  if (r === 0) return "∅"; // Symbol angepasst für leere Menge

  // epsilon
  if (r === "ε" || r === "𝜀") return "ε";

  // einzelnes Symbol
  if (typeof r === "string") return r;

  // Array-Fälle
  if (Array.isArray(r)) {
    // -----------------------------------------
    // Binäre Operatoren: [r1, '⋅', r2] / [r1, '+', r2]
    // -----------------------------------------
    if (r.length === 3) {
      const [r1, op, r2] = r as [RegExp, string, RegExp];

      if (op === "⋅") {
        // Bei Konkatenation Klammern nur setzen, wenn nötig (hier vereinfacht immer ohne, außer bei +)
        // Für eine korrekte Operator-Präzedenz wäre eine komplexere Logik nötig.
        // Hier nutzen wir eine einfache Version.
        const s1 = regexpToString(r1);
        const s2 = regexpToString(r2);
        
        // Wenn r1 oder r2 eine Summe (+) ist, sollte man klammern, aber ⋅ bindet stärker.
        // Einfache Heuristik:
        const p1 = Array.isArray(r1) && r1[1] === '+' ? `(${s1})` : s1;
        const p2 = Array.isArray(r2) && r2[1] === '+' ? `(${s2})` : s2;
        
        return p1 + p2; // Punkt oft weglassen oder explizit: p1 + "⋅" + p2
      }

      if (op === "+") {
        return regexpToString(r1) + "+" + regexpToString(r2);
      }
    }

    // -----------------------------------------
    // Kleene-Stern: [expr, '*']
    // -----------------------------------------
    if (r.length === 2 && r[1] === "*") {
      const inner = r[0] as RegExp;
      const sInner = regexpToString(inner);

      // Symbol* oder (Ausdruck)*
      if (typeof inner === "string" || typeof inner === "number") {
        return sInner + "*";
      }

      return "(" + sInner + ")*";
    }
  }

  return JSON.stringify(r);
}

In [ ]:
console.log("\nEndgültiger regulärer Ausdruck (String):");
console.log(regexpToString(s));